# Vamos a leer el cluster 2 y generar un dataset sencillo con LGBM

vamos por paso

En este caso no voy a generar las covariables que generó Daiana

Voy a probar que corra, si corre luego le agrego las variables y veo los cambios en Kaggle




## Preparo todo

In [1]:
### me preparo en el entorno
# entorno
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    #FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    #DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
    FUENTES_DIR = '/content/drive/MyDrive/MCD/Labo3/FUENTES/' # Ajustar según usuario
    DATOS_DIR = '/content/drive/MyDrive/MCD_grupo17_la_desgracia/lab3/PruebasAK/DATOSL3/' # ajustar según usuario OJO crear un acceso directo para poder "montar" una unidad compartida
else:
    # configuración para notebook con instalación LOCAL
    # FUENTES_DIR = '../FUENTES'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = './DATOS/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Mounted at /content/drive/


In [2]:
## Cargo lo necesario
# Cargo paquetes
#Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from numpy.polynomial import Polynomial
#from scipy.signal import detrend



#import holidays

#Configuración de parametros de Pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('plotting.backend', 'matplotlib')

In [3]:
from sklearn.model_selection import train_test_split, TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
import lightgbm as lgb
import matplotlib.pyplot as plt



## Levanto el dataset
vamos a hacer el promedio de los últimos 12 meses por producto

In [4]:
# me traigo el archivo ya procesado con la duración completa
df_cluster_1 = pd.read_parquet(DATOS_DIR +'df_cluster_1.parquet')
#df_cluster_3.head()
# productos a validar Son los que se van a estar vivos
productos_validar = pd.read_csv(DATOS_DIR +'product_id_apredecir201912.txt', sep='\t')
# me traigo el clasificador de clusters


In [5]:
# me quedo con las filas de df_cluster_3 cuyo product_id esté en productos_validar
###################################################
##### OJO, en realidad debo dejar todo
##### dejo las otras series que ayuden
##### saco de todo el conjunto lo que necesito al final
#############################
#df_cluster_X = df_cluster_3[df_cluster_3['product_id'].isin(productos_validar['product_id'])]
df_cluster_X = df_cluster_1.copy()
df_cluster_X.shape

(236511, 45)

In [6]:
df_cluster_X.shape

(236511, 45)

In [7]:
df_cluster_X.dtypes


,0
customer_id,int64
product_id,int64
date_mes,period[M]
first_customer_date,period[M]
last_customer_date,period[M]
first_product_date,period[M]
last_product_date,period[M]
tn,float64
dif_tn,float64
dif_por,float64


In [8]:
# tengo que pasar a timestamp todas las columnas con formato period[M] de df_cluster_X
# Identificamos automáticamente las columnas de tipo Period[M]
period_cols = df_cluster_X.select_dtypes(include=['period[M]']).columns

# Convertimos todas esas columnas a datetime usando .dt.to_timestamp()
df_cluster_X[period_cols] = df_cluster_X[period_cols].apply(lambda col: col.dt.to_timestamp())
df_cluster_X.dtypes

,0
customer_id,int64
product_id,int64
date_mes,datetime64[ns]
first_customer_date,datetime64[ns]
last_customer_date,datetime64[ns]
first_product_date,datetime64[ns]
last_product_date,datetime64[ns]
tn,float64
dif_tn,float64
dif_por,float64


In [ ]:
###### POR AHORA NO LO EJECUTO###############
##### Creo variables, en este caso como Daiana


## Ajustes propios de autogluon

#### known covariantes


## agrando el df con lo realizado por Daiana / CHAT

In [9]:
df_cluster_X.columns

Index(['customer_id', 'product_id', 'date_mes', 'first_customer_date',
       'last_customer_date', 'first_product_date', 'last_product_date', 'tn',
       'dif_tn', 'dif_por', 'vida_prod', 'vida_customer', 'meses_on',
       'meses_off', 'vida_restante', 'anio', 'mes', 'secuencia', 'trimestre',
       'cat1', 'cat1_cat2', 'cat1_cat2_cat3', 'cat1_cat2_cat3_brand',
       'cat1_cat2_cat3_brand_size', 'periodo', 'stock_final',
       'dias_laborables_arg', 'cisne_negro', 'VtaGral', 'VtaGralCte',
       'Bebidas', 'Almacén', 'Panadería', 'Lácteos', 'Carnes',
       'Verdulería y frutería', 'Artículos de limpieza y perfumería',
       'Indumentaria, calzado y textiles para el hogar',
       'Electrónicos y artículos para el hogar', 'Otros', 'USD_of', 'USD_blue',
       'Infl', 'Expect', 'product_cust_id'],
      dtype='object')

In [10]:
# 3. Media móvil de los últimos 3 meses
df_cluster_X['media_movil_3'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(
    lambda x: x.shift(2).rolling(window=3, min_periods=1).mean()
)

In [11]:
# 3. Media móvil de los últimos 6 meses
df_cluster_X['media_movil_6'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(
    lambda x: x.shift(2).rolling(window=6, min_periods=1).mean()
)

# 3. Media móvil de los últimos 9 meses
df_cluster_X['media_movil_9'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(
    lambda x: x.shift(2).rolling(window=9, min_periods=1).mean()
)

# 3. Media móvil de los últimos 12 meses
df_cluster_X['media_movil_12'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(
    lambda x: x.shift(2).rolling(window=12, min_periods=1).mean()
)

# Diferencia entre t-2 y t-3 (omitimos t y t-1)
df_cluster_X['delta_tn_1m'] = (
    df_cluster_X
    .groupby('product_cust_id')['tn']
    .transform(lambda x: x.shift(2) - x.shift(3))
)

In [12]:
# 5. Volatilidad (desvío estándar de los últimos 3 meses)
df_cluster_X['std_tn_3m'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(
    lambda x: x.shift(2).rolling(window=3, min_periods=1).std()
)

# 5. Volatilidad (desvío estándar de los últimos 6 meses)
df_cluster_X['std_tn_6m'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(
    lambda x: x.shift(2).rolling(window=6, min_periods=1).std()
)


# 7 Baja en t-2, t-3, t-4 (sin usar t ni t-1)
df_cluster_X['caida_reciente'] = (
    df_cluster_X
    .groupby('product_cust_id')['tn']
    .transform(lambda x: (x.shift(2) < x.shift(3)) & (x.shift(3) < x.shift(4)))
    .fillna(False)
    .astype(int)
)

# 8 Alza en t-2, t-3, t-4 (sin usar t ni t-1)
df_cluster_X['alza_reciente'] = (
    df_cluster_X
    .groupby('product_cust_id')['tn']
    .transform(lambda x: (x.shift(2) > x.shift(3)) & (x.shift(3) > x.shift(4)))
    .fillna(False)
    .astype(int)
)

In [13]:


# 9 Pendiente lineal (rolling trend)
def rolling_slope(x, window=4):
    return x.rolling(window).apply(lambda y: np.polyfit(range(len(y)), y, 1)[0], raw=False)

df_cluster_X['trend_4'] = df_cluster_X.groupby('product_cust_id')['tn'].transform(lambda x: rolling_slope(x.shift(2), window=4))

#10 mes con mayor venta historica
peak_month = df_cluster_X.groupby('product_cust_id')['tn'].idxmax().apply(lambda i: df_cluster_X.loc[i, 'date_mes'].month)
df_cluster_X = df_cluster_X.merge(peak_month.rename('mes_pico'), on='product_cust_id')
df_cluster_X['es_mes_pico'] = (df_cluster_X['date_mes'].dt.month == df_cluster_X['mes_pico']).astype(int)

# Variable: meses desde el máximo en los últimos 6 meses (sin contar t ni t+1)
def meses_desde_max_6m(grupo):
    resultados = []
    for i in range(len(grupo)):
        # Tomar los 6 meses anteriores a t-1 (es decir, desde t-2 a t-7)
        ventana = grupo.iloc[max(0, i - 7):max(0, i - 1)]
        if ventana.empty or ventana.isna().all():
            resultados.append(np.nan)
            continue
        max_valor = ventana.max()
        # Buscar el índice (posición) del último máximo dentro de la ventana
        idx_max = ventana[::-1].idxmax()
        meses_pasados = i - grupo.index.get_loc(idx_max)
        resultados.append(meses_pasados)
    return pd.Series(resultados, index=grupo.index)

df_cluster_X['meses_desde_max_6m'] = (
    df_cluster_X.groupby('product_cust_id')['tn']
    .apply(meses_desde_max_6m)
    .reset_index(level=0, drop=True)  # Alinear con df_cluster_X
)


In [14]:
# Venta total (por mes y cada nivel de categoría)
category_levels = [
    'cat1', 'cat1_cat2', 'cat1_cat2_cat3', 'cat1_cat2_cat3_brand', 'cat1_cat2_cat3_brand_size'
]

for cat in category_levels:
    # Total ventas por mes y categoria
    df_cluster_X[f'Vta_total_{cat}'] = df_cluster_X.groupby(['date_mes', cat])['tn'].transform('sum')
    # Rank dentro de categoría por mes (mayor tn = ranking 1)
    df_cluster_X[f'rank_{cat}'] = df_cluster_X.groupby(['date_mes', cat])['tn'].rank(method='first', ascending=False)
    # Participación (share)
    df_cluster_X[f'share_{cat}'] = df_cluster_X['tn'] / df_cluster_X[f'Vta_total_{cat}']

In [15]:
df_cluster_X.shape

(236511, 73)

## ahora normalizo

In [16]:
# --- Paso 1: Normalización robusta con clase para reusabilidad --- #
class RobustScalerCustom(BaseEstimator, TransformerMixin):
    """
    Escala series con std si std !=0, sino con media.
    Guarda parámetros para poder revertir la normalización.
    """
    def fit(self, X, y=None):
        self.mean_ = X.mean()
        self.std_ = X.std()
        if self.std_ == 0:
            self.std_ = None
            if self.mean_ == 0:
                self.mean_ = 1
        return self
    def transform(self, X):
        if self.std_ is not None:
            return (X - self.mean_) / self.std_
        else:
            return X / self.mean_
    def inverse_transform(self, X_scaled):
        if self.std_ is not None:
            return X_scaled * self.std_ + self.mean_
        else:
            return X_scaled * self.mean_

In [17]:
# lgbm no banca tildes ni espacios en los nombres de columna
# Diccionario de reemplazos para vocales con tilde
replacements = {
    'á': 'a',
    'é': 'e',
    'í': 'i',
    'ó': 'o',
    'ú': 'u',
    'Á': 'A',
    'É': 'E',
    'Í': 'I',
    'Ó': 'O',
    'Ú': 'U',
}

def clean_column_names(col_names):
    cleaned_cols = []
    for col in col_names:
        # Reemplazar espacios por guion bajo
        col = col.replace(' ', '_')
        # Eliminar comas
        col = col.replace(',', '')
        # Reemplazar vocales con tilde
        for orig, repl in replacements.items():
            col = col.replace(orig, repl)
        cleaned_cols.append(col)
    return cleaned_cols

# Aplica al DataFrame df_cluster_X
df_cluster_X.columns = clean_column_names(df_cluster_X.columns)

# Comprobar resultado
print(df_cluster_X.columns)

Index(['customer_id', 'product_id', 'date_mes', 'first_customer_date',
       'last_customer_date', 'first_product_date', 'last_product_date', 'tn',
       'dif_tn', 'dif_por', 'vida_prod', 'vida_customer', 'meses_on',
       'meses_off', 'vida_restante', 'anio', 'mes', 'secuencia', 'trimestre',
       'cat1', 'cat1_cat2', 'cat1_cat2_cat3', 'cat1_cat2_cat3_brand',
       'cat1_cat2_cat3_brand_size', 'periodo', 'stock_final',
       'dias_laborables_arg', 'cisne_negro', 'VtaGral', 'VtaGralCte',
       'Bebidas', 'Almacen', 'Panaderia', 'Lacteos', 'Carnes',
       'Verduleria_y_fruteria', 'Articulos_de_limpieza_y_perfumeria',
       'Indumentaria_calzado_y_textiles_para_el_hogar',
       'Electronicos_y_articulos_para_el_hogar', 'Otros', 'USD_of', 'USD_blue',
       'Infl', 'Expect', 'product_cust_id', 'media_movil_3', 'media_movil_6',
       'media_movil_9', 'media_movil_12', 'delta_tn_1m', 'std_tn_3m',
       'std_tn_6m', 'caida_reciente', 'alza_reciente', 'trend_4', 'mes_pico',
      

In [18]:
# --- Paso 3: Selección columnas a normalizar y preparación de scalers --- #
features_to_normalize = [
    'tn','dif_tn','dif_por','vida_prod','vida_customer','meses_on','meses_off','vida_restante','stock_final',
    'VtaGral','VtaGralCte','Bebidas','Almacen','Panaderia','Lacteos','Carnes','Verduleria_y_fruteria',
    'Articulos_de_limpieza_y_perfumeria','Indumentaria_calzado_y_textiles_para_el_hogar',
    'Electronicos_y_articulos_para_el_hogar','Otros','USD_of','USD_blue','Infl','Expect',
    'media_movil_3','media_movil_6','media_movil_9','media_movil_12','delta_tn_1m',
    'std_tn_3m','std_tn_6m','caida_reciente','alza_reciente','trend_4','mes_pico','es_mes_pico','meses_desde_max_6m','Vta_total_cat1', 'rank_cat1',
    'share_cat1', 'Vta_total_cat1_cat2', 'rank_cat1_cat2',
    'share_cat1_cat2', 'Vta_total_cat1_cat2_cat3', 'rank_cat1_cat2_cat3',
    'share_cat1_cat2_cat3', 'Vta_total_cat1_cat2_cat3_brand',
    'rank_cat1_cat2_cat3_brand', 'share_cat1_cat2_cat3_brand',
    'Vta_total_cat1_cat2_cat3_brand_size', 'rank_cat1_cat2_cat3_brand_size',
    'share_cat1_cat2_cat3_brand_size'
]

# Creamos un diccionario para guardar scalers independientes por variable
scalers_dict = {}

for col in features_to_normalize:
    scaler = RobustScalerCustom()
    scaler.fit(df_cluster_X[col])
    df_cluster_X[f'norm_{col}'] = scaler.transform(df_cluster_X[col])
    scalers_dict[col] = scaler

In [19]:
# --- Paso 4: Preparar target a predecir: tn(t+2) --- #
df_cluster_X = df_cluster_X.sort_values(['product_cust_id', 'date_mes']).reset_index(drop=True)
df_cluster_X['tn_plus_2'] = df_cluster_X.groupby('product_cust_id')['tn'].shift(-2)
# Drop filas sin target (los dos últimos de cada serie)
df_model = df_cluster_X.dropna(subset=['tn_plus_2']).reset_index(drop=True)

# --- Paso 5: Normalizamos target usando scaler de 'tn' --- #
scaler_target = scalers_dict['tn']
df_model['norm_tn_plus_2'] = scaler_target.transform(df_model['tn_plus_2'])

In [20]:
df_cluster_X.columns

Index(['customer_id', 'product_id', 'date_mes', 'first_customer_date',
       'last_customer_date', 'first_product_date', 'last_product_date', 'tn',
       'dif_tn', 'dif_por',
       ...
       'norm_Vta_total_cat1_cat2_cat3', 'norm_rank_cat1_cat2_cat3',
       'norm_share_cat1_cat2_cat3', 'norm_Vta_total_cat1_cat2_cat3_brand',
       'norm_rank_cat1_cat2_cat3_brand', 'norm_share_cat1_cat2_cat3_brand',
       'norm_Vta_total_cat1_cat2_cat3_brand_size',
       'norm_rank_cat1_cat2_cat3_brand_size',
       'norm_share_cat1_cat2_cat3_brand_size', 'tn_plus_2'],
      dtype='object', length=127)

In [21]:
# Columnas estáticas, categóricas o numéricas (ajustar al caso real):
static_cols = [
    "customer_id","product_id","cat1","cat1_cat2","cat1_cat2_cat3",
    "cat1_cat2_cat3_brand","cat1_cat2_cat3_brand_size"
]
extern_cols = [
    "dias_laborables_arg","anio","mes","secuencia","trimestre","cisne_negro","periodo"
]

In [22]:
feature_cols = extern_cols + static_cols
feature_cols += [f'norm_{c}' for c in features_to_normalize if f'norm_{c}' in df_model.columns]
feature_cols += [f'Vta_total_{cat}' for cat in category_levels]
feature_cols += [f'rank_{cat}' for cat in category_levels]
feature_cols += [f'share_{cat}' for cat in category_levels]

# Codificamos categorías con Label Encoding para LightGBM (requiere convertir a entero)
for col in static_cols + extern_cols:
    if col in df_model.columns and df_model[col].dtype == 'object':
        df_model[col] = df_model[col].astype('category').cat.codes

X = df_model[feature_cols]
y = df_model['norm_tn_plus_2']

In [23]:
print(feature_cols) # no debiera tener codiogs raros

['dias_laborables_arg', 'anio', 'mes', 'secuencia', 'trimestre', 'cisne_negro', 'periodo', 'customer_id', 'product_id', 'cat1', 'cat1_cat2', 'cat1_cat2_cat3', 'cat1_cat2_cat3_brand', 'cat1_cat2_cat3_brand_size', 'norm_tn', 'norm_dif_tn', 'norm_dif_por', 'norm_vida_prod', 'norm_vida_customer', 'norm_meses_on', 'norm_meses_off', 'norm_vida_restante', 'norm_stock_final', 'norm_VtaGral', 'norm_VtaGralCte', 'norm_Bebidas', 'norm_Almacen', 'norm_Panaderia', 'norm_Lacteos', 'norm_Carnes', 'norm_Verduleria_y_fruteria', 'norm_Articulos_de_limpieza_y_perfumeria', 'norm_Indumentaria_calzado_y_textiles_para_el_hogar', 'norm_Electronicos_y_articulos_para_el_hogar', 'norm_Otros', 'norm_USD_of', 'norm_USD_blue', 'norm_Infl', 'norm_Expect', 'norm_media_movil_3', 'norm_media_movil_6', 'norm_media_movil_9', 'norm_media_movil_12', 'norm_delta_tn_1m', 'norm_std_tn_3m', 'norm_std_tn_6m', 'norm_caida_reciente', 'norm_alza_reciente', 'norm_trend_4', 'norm_mes_pico', 'norm_es_mes_pico', 'norm_meses_desde_max_

In [24]:
# --- Paso 7: División temporal para evitar filtración --- #
split = int(len(X) * 0.8)
X_train, X_val = X.iloc[:split], X.iloc[split:]
y_train, y_val = y.iloc[:split], y.iloc[split:]

In [25]:
param_dist = {
    'num_leaves': [31, 50, 70, 100],
    'max_depth': [5, 10, 20, -1],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 200],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
}

lgb_estimator = lgb.LGBMRegressor(random_state=42)

tscv = TimeSeriesSplit(n_splits=3)

In [26]:
random_search = RandomizedSearchCV(
    estimator=lgb_estimator,
    param_distributions=param_dist,
    n_iter=10,# dejar luego en 30
    cv=tscv,
    scoring='neg_root_mean_squared_error',
    verbose=3,# cambiar a 3 voy a cortar en una hora si no tengo nada mejor
    random_state=42,
    n_jobs=-1,
)

random_search.fit(X_train, y_train)

print("Mejores parametros:", random_search.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014656 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11445
[LightGBM] [Info] Number of data points in the train set: 177186, number of used features: 82
[LightGBM] [Info] Start training from score 0.026318
Mejores parametros: {'subsample': 0.6, 'num_leaves': 70, 'n_estimators': 50, 'min_child_samples': 20, 'max_depth': 10, 'learning_rate': 0.05, 'colsample_bytree': 1.0}


 {'subsample': 0.6, 'num_leaves': 70, 'n_estimators': 50, 'min_child_samples': 20, 'max_depth': 10, 'learning_rate': 0.05, 'colsample_bytree': 1.0}

In [27]:
# --- Paso 9: Modelo final con mejores parametros --- #
best_params = random_search.best_params_
model = lgb.LGBMRegressor(**best_params, random_state=42)
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014653 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11445
[LightGBM] [Info] Number of data points in the train set: 177186, number of used features: 82
[LightGBM] [Info] Start training from score 0.026318


LGBMRegressor(learning_rate=0.05, max_depth=10, n_estimators=50, num_leaves=70,
              random_state=42, subsample=0.6)

In [28]:
# Predicción y desfase inverso para interpretar salida ---
y_val_pred = model.predict(X_val)
# Transformamos predicciones de vuelta a escala original
y_val_pred_original = scaler_target.inverse_transform(y_val_pred)
y_val_original = scaler_target.inverse_transform(y_val.values)

mse_val = mean_squared_error(y_val_original, y_val_pred_original)  # solo MSE
rmse_val = np.sqrt(mse_val)  # RMSE es raíz cuadrada de MSE
print(f"RMSE validación (escala original): {rmse_val:.4f}")

RMSE validación (escala original): 0.5421


In [29]:
# --- Paso 10: Forecast a futuro para el último dato de cada product_cust_id --- #
# Tomar los últimos registros de cada serie para predecir tn+2
to_forecast = df_cluster_X.groupby('product_cust_id').tail(1)  # último punto (de algún modo, adaptar según sea necesario)

# Preprocesar features igual que antes
for col in static_cols + extern_cols:
    if col in to_forecast.columns and to_forecast[col].dtype == 'object':
        to_forecast[col] = to_forecast[col].astype('category').cat.codes
for col in features_to_normalize:
    col_norm = f'norm_{col}'
    if col_norm not in to_forecast.columns:
        to_forecast[col_norm] = scalers_dict[col].transform(to_forecast[col])

X_forecast = to_forecast[feature_cols]
forecast_norm = model.predict(X_forecast)
forecast = scaler_target.inverse_transform(forecast_norm)

resultados = to_forecast[['product_cust_id', 'date_mes']].copy()
resultados['forecast_tn_plus_2'] = forecast

print(resultados.head())

    product_cust_id   date_mes  forecast_tn_plus_2
35      20001_10002 2019-12-01              72.142
71      20001_10017 2019-12-01              30.560
107     20001_10019 2019-12-01              13.928
143     20001_10021 2019-12-01              28.957
179     20001_10022 2019-12-01              10.712


/tmp/ipython-input-29-1424922972.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_forecast[col] = to_forecast[col].astype('category').cat.codes
/tmp/ipython-input-29-1424922972.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_forecast[col] = to_forecast[col].astype('category').cat.codes
/tmp/ipython-input-29-1424922972.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [30]:
resultados.shape

(7514, 3)

In [31]:
resultados.head()

,product_cust_id,date_mes,forecast_tn_plus_2
35,20001_10002,2019-12-01,72.142
71,20001_10017,2019-12-01,30.560
107,20001_10019,2019-12-01,13.928
143,20001_10021,2019-12-01,28.957
179,20001_10022,2019-12-01,10.712


### maniobrando para volver

## manioboro para sacar la predicción

In [32]:
# creo la columna product_id como los cinco primeros caracteres de la columna item_id de df_pred
df_pred = resultados.copy()
df_pred['product_id'] = df_pred['product_cust_id'].str[:5]
df_pred.head()

,product_cust_id,date_mes,forecast_tn_plus_2,product_id
35,20001_10002,2019-12-01,72.142,20001
71,20001_10017,2019-12-01,30.560,20001
107,20001_10019,2019-12-01,13.928,20001
143,20001_10021,2019-12-01,28.957,20001
179,20001_10022,2019-12-01,10.712,20001


In [33]:
# obtengo df_pred1 como la suma de mean para cada timestamp y cada product_id
df_pred1 = df_pred.groupby(['product_id', 'date_mes'])['forecast_tn_plus_2'].sum().reset_index()
df_pred1.head()

,product_id,date_mes,forecast_tn_plus_2
0,20001,2019-12-01,280.097
1,20002,2019-12-01,557.571
2,20003,2019-12-01,229.885
3,20004,2019-12-01,230.187
4,20005,2019-12-01,227.234


In [34]:
# llevo la columna timestamp a formato period[M]
df_pred1['date_mes'] = pd.to_datetime(df_pred1['date_mes'])
df_pred1['date_mes'] = df_pred1['date_mes'].dt.to_period('M')
df_pred1.head()

,product_id,date_mes,forecast_tn_plus_2
0,20001,2019-12,280.097
1,20002,2019-12,557.571
2,20003,2019-12,229.885
3,20004,2019-12,230.187
4,20005,2019-12,227.234


In [35]:
df_pred1.shape

(692, 3)

In [36]:
df_pred1.dtypes

,0
product_id,object
date_mes,period[M]
forecast_tn_plus_2,float64


In [37]:
#llevo la columna product_id de df_pred1 a integrer
df_pred1['product_id'] = df_pred1['product_id'].astype(int)
df_pred1.dtypes

,0
product_id,int64
date_mes,period[M]
forecast_tn_plus_2,float64


In [38]:
productos_validar.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


In [39]:
productos_validar.shape

(780, 1)

In [40]:
productos_validar.dtypes

,0
product_id,int64


In [41]:
# Merge para obtener el promedio solo para los product_id que están en productos_validar
df_validar = productos_validar.merge(
    df_pred1,
    on='product_id',
    how='left'
)

# Ahora df_validar_promedio tendrá dos columnas: product_id y tn_prom_2018
print(df_validar.head())

   product_id date_mes  forecast_tn_plus_2
0       20001  2019-12             280.097
1       20002  2019-12             557.571
2       20003  2019-12             229.885
3       20004  2019-12             230.187
4       20005  2019-12             227.234


In [42]:
df_validar.tail()

,product_id,date_mes,forecast_tn_plus_2
775,21263,NaT,NaN
776,21265,NaT,NaN
777,21266,NaT,NaN
778,21267,NaT,NaN
779,21276,NaT,NaN


In [43]:
df_validar.shape

(780, 3)

In [44]:
# llevo al formato deseado, dropeo timestamp
df_validar_promedio = df_validar.drop(columns=['date_mes'])
df_validar_promedio.head()

,product_id,forecast_tn_plus_2
0,20001,280.097
1,20002,557.571
2,20003,229.885
3,20004,230.187
4,20005,227.234


In [45]:
# le cambio el nombre de mean a tn
df_validar_promedio.rename(columns={'forecast_tn_plus_2': 'tn'}, inplace=True)
df_validar_promedio.head()

,product_id,tn
0,20001,280.097
1,20002,557.571
2,20003,229.885
3,20004,230.187
4,20005,227.234


In [46]:
df_validar_promedio.to_csv(DATOS_DIR +'df_cluster1_lgbm_V1.csv', index=False)